In [1]:
import pandas as pd
from ax import *

import numpy as np

from ax.core.metric import Metric
from ax.metrics.noisy_function import NoisyFunctionMetric
from ax.service.utils.report_utils import exp_to_df
from ax.runners.synthetic import SyntheticRunner

# Factory methods for creating multi-objective optimization model.
from ax.modelbridge.factory import get_MOO_EHVI, get_MOO_PAREGO

# Analysis utilities, including a method to evaluate hypervolumes
from ax.modelbridge.modelbridge_utils import observed_hypervolume
from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties

import torch
from ax.utils.common.result import Ok, Err
# Plotting imports and initialization
from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.plot.pareto_utils import compute_posterior_pareto_frontier
from ax.plot.pareto_frontier import plot_pareto_frontier
init_notebook_plotting()


: 

: 

In [2]:
# Define all of the parameter
featureNumber = 12
DomainPath = '../Data/20230207_Summary.csv'
f = open(DomainPath, 'r',encoding='utf-8-sig')
RawData = np.genfromtxt(f, delimiter=",")
N_BATCH = 6

In [3]:
##Merge the candidates with the predicted values
def prediction_result_output(candidates)->pd.DataFrame:
    output=candidates.param_df.round(4)
    prediction = pd.DataFrame(candidates.model_predictions_by_arm)
    prediction = prediction.transpose()
    mean_list = []
    covar_list = []
    #Match the signature with the mean and covar
    for i in range(prediction.index.__len__()):
        prediction[0][i] = prediction[0][i]['GPx_metric']
        prediction[1][i] = prediction[1][i]['GPx_metric']['GPx_metric']
        if (output.index[i] in prediction.index[i]):
            mean_list.append(prediction.iloc[i , 0])
            covar_list.append(prediction.iloc[i , 1])
    output['Predict_Mean'] = mean_list
    output['Predict_Covar'] = covar_list
    return output

In [4]:
# Retrive the data from CSV
class IFN_data(NoisyFunctionMetric):
    def f(self, x: np.ndarray) -> float:
        global RawData
        activity = 0
        print('x_IFN=',x)
        for i in range(RawData.shape[0]):
            #label the fetched data
            if ((RawData[i, :featureNumber] == x).all()) and (RawData[i, featureNumber+3] == 0):
                RawData[i, featureNumber+3] = 1
                activity = RawData[i, featureNumber+1]
                print(activity)
        return float(activity)
    
class GFP_data(NoisyFunctionMetric):
    def f(self, x: np.ndarray) -> float:
        global RawData
        activity = 0 
        print('x_=',x)
        for i in range(RawData.shape[0]):
            #label the fetched data
            if ((RawData[i, :featureNumber] == x).all()) and (RawData[i, featureNumber+2] == 0):
                RawData[i, featureNumber+2] = 1
                activity = RawData[i, featureNumber]
                print(activity)
        return float(activity)


In [5]:
metric_IFN = IFN_data("IFN", [f"x{i+1}" for i in range(featureNumber)], noise_sd=0.0, lower_is_better=False)
metric_GFP = GFP_data("GFP", [f"x{i+1}" for i in range(featureNumber)], noise_sd=0.0, lower_is_better=True)
mo = MultiObjective(
    objectives=[Objective(metric=metric_IFN), Objective(metric=metric_GFP)],
)

# set the reference point as (4,4)
objective_thresholds = [
    ObjectiveThreshold(metric=metric, bound=val, relative=False)
    for metric, val in [(metric_IFN,2),(metric_GFP,2)]
]

optimization_config = MultiObjectiveOptimizationConfig(
    objective=mo,
    objective_thresholds=objective_thresholds,
)

In [6]:
# Define the search space
MOO_search_space = SearchSpace(
    parameters = [
        RangeParameter(
            name=f"x{i+1}", parameter_type=ParameterType.FLOAT, lower=0.0, upper=100.0
        )
        for i in range(featureNumber)
    ]
)
# Set Sum constraints 
sum_constraint_upper = SumConstraint(
    parameters = [MOO_search_space.parameters[f'x{i+1}'] for i in range(featureNumber)],
    # parameters = list(GPx_search_space.parameters.values()),
    # parameters = [GPx_search_space.parameters[f'x{i}'] for i in range(1, 8)]),
    is_upper_bound = True,
    bound = 101,
)
sum_constraint_lower = SumConstraint(
    parameters = [MOO_search_space.parameters[f'x{i+1}'] for i in range(featureNumber)],
    is_upper_bound = False,
    bound = 99,
)
MOO_search_space.add_parameter_constraints([sum_constraint_upper])
MOO_search_space.add_parameter_constraints([sum_constraint_lower])

In [7]:
def build_experiment():
    experiment = Experiment(
        name="pareto_experiment",
        search_space=MOO_search_space,
        optimization_config=optimization_config,
        runner=SyntheticRunner(),
    )
    return experiment
## Initialize with selected points

def initialize_experiment(experiment):
    b = []
    print('Fetching data...')
    for i in range(0,RawData.shape[0]):
        a = [Arm(parameters = {f'x{x+1}': RawData[i,x] for x in range(featureNumber)})]
        b=b+a
    gr = GeneratorRun(b)
    experiment.new_batch_trial(generator_run=gr).run()
    return experiment.fetch_data()

In [8]:
ehvi_experiment = build_experiment()
ehvi_data = initialize_experiment(ehvi_experiment)
ehvi_data

Fetching data...
x_IFN= [ 5.8  0.4 13.1 11.4  3.5  8.7  8.5 13.9  8.3  7.3  8.8 10.4]
0.121
x_IFN= [10.6  7.8  5.9 11.9 17.7  2.9  9.3  1.  13.6  4.4  3.1 11.9]
0.776
x_IFN= [ 0.4  6.2 11.1  5.5  6.6 17.7  1.6 11.8  1.8 17.9  1.4 17.9]
0.119
x_IFN= [ 3.2  4.1 18.7 19.7  6.8  0.9  1.3  5.   1.1 17.4  1.  20.7]
0.809
x_IFN= [ 9.7  4.7 10.6 14.5  4.9  5.4  5.2  9.1  8.7 14.3 11.2  1.6]
0.081
x_IFN= [16.7  4.1  7.6 10.5 12.  13.9  6.9  3.1 12.1  7.1  0.4  5.6]
0.066
x_IFN= [ 4.1  2.2  0.1  2.2  4.3 16.1  6.5  8.9 14.2  7.4 16.8 17.2]
0.251
x_IFN= [ 4.5 15.3 15.   3.8 11.3  6.   2.3 10.3  2.4 15.1  6.9  7. ]
0.162
x_IFN= [ 0.2  7.3  3.3 17.1  5.9 19.7  8.8 17.2  1.9  1.5  4.  13.1]
0.029
x_IFN= [17.6 16.1 17.4  3.7  3.6  0.9 15.6  0.1  6.1  1.7 10.3  6.9]
2.204
x_IFN= [ 7.5  5.8  3.6 18.8  2.4 14.4 16.8  7.5  3.5  4.   3.  12.7]
0.084
x_IFN= [ 9.4  1.1  5.3  5.6 14.8 12.6  8.1  4.9  3.  12.5 14.2  8.5]
0.108
x_IFN= [13.3  0.3 15.2 10.6  4.4  3.6  0.7  9.  16.9 13.4 11.4  1.1]
0.169
x_IFN= [

In [13]:
ehvi_hv_list = []
ehvi_model = None
ehvi_model = get_MOO_EHVI(
        experiment=ehvi_experiment, 
        data=ehvi_data,
    )
hv = observed_hypervolume(modelbridge=ehvi_model)

/home/guangqi/miniconda3/envs/pkuBO/lib/python3.9/site-packages/ax/modelbridge/modelbridge_utils.py:811: UserWarning:

FYI: The default behavior of `get_pareto_frontier_and_configs` when `transform_outcomes_and_configs` is not specified has changed. Previously, the default was `transform_outcomes_and_configs=True`; now this argument is deprecated and behavior is as if `transform_outcomes_and_configs=False`. You did not specify `transform_outcomes_and_configs`, so this warning requires no action.



In [14]:
hv

0.2210460000000001

In [16]:
c = ehvi_model.gen(20)

/home/guangqi/miniconda3/envs/pkuBO/lib/python3.9/site-packages/botorch/optim/optimize.py:306: RuntimeWarning:

Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 9.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 9.')]
Trying again with a new set of initial conditions.

/home/guangqi/miniconda3/envs/pkuBO/lib/python3.9/site-packages/botorch/optim/optimize.py:328: RuntimeWarning:

Optimization failed on the second try, after generating a new set of initial conditions.

/home/guangqi/miniconda3/envs/pkuBO/lib/python3.9/site-packages/botorch/optim/optimize.py:306: RuntimeWarning:

Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 9.')]
Trying again with a new set of initial conditions.

/home/guangqi/miniconda3/envs/pkuBO/li

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.75 GiB (GPU 0; 23.70 GiB total capacity; 14.80 GiB already allocated; 1.79 GiB free; 20.65 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [16]:
c.param_df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12
1417,27.230789,2.462959e+01,3.201253,1.281252,6.333694e-15,3.734172e+00,1.829763,1.020700e-14,1.078614,6.014609e-15,0.000000e+00,38.014562
4f49,31.885471,8.043234e-15,8.918904,2.405823,0.000000e+00,2.496325e+00,21.633694,6.948205e-15,2.494002,1.190211e-14,0.000000e+00,31.165781
fc1f,0.000000,3.278495e+01,8.068227,3.114942,1.165104e+00,2.316173e+00,22.776335,1.125755e+00,2.557129,0.000000e+00,1.556058e-12,27.091388
b620,12.425729,7.867273e-01,8.973555,6.194199,0.000000e+00,0.000000e+00,24.923897,0.000000e+00,13.688210,2.029055e+01,2.042643e+00,9.674491
fcd4,40.672263,0.000000e+00,8.973672,6.224982,0.000000e+00,5.699401e-13,23.110996,1.885657e-13,13.699966,0.000000e+00,0.000000e+00,8.318121
65b3,0.821383,3.134191e+00,8.764637,5.847927,0.000000e+00,0.000000e+00,26.678746,0.000000e+00,13.692991,2.908303e+01,1.871928e+00,9.105163
cf0a,23.335272,1.028463e+01,8.645118,5.920767,2.695436e-10,0.000000e+00,26.463511,0.000000e+00,13.712942,9.337175e-11,9.884936e-11,10.637759
84c6,10.523331,1.696378e+00,9.232612,6.090170,0.000000e+00,0.000000e+00,28.272449,0.000000e+00,13.687563,5.297239e-01,1.842365e+01,10.544122
c237,36.372266,1.148506e-12,8.522788,6.077106,6.959292e-12,1.109153e-12,19.643440,7.423072e-12,13.700156,1.338082e+01,1.844702e-11,3.303428
43ea,22.561542,7.605420e-01,8.387260,5.907072,0.000000e+00,0.000000e+00,24.416905,1.538295e-16,13.707074,1.630812e+01,0.000000e+00,8.951491


In [17]:
c.model_predictions_by_arm

{'dc26f15c201505a7b3cbc0adc76d1417': ({'IFN': 2.967678892199969,
   'GFP': 2.482944774496509},
  {'IFN': {'IFN': 0.29188142265038813, 'GFP': 0.0},
   'GFP': {'IFN': 0.0, 'GFP': 0.349202071229788}}),
 'e6c69d3b709ccabbe04b911880534f49': ({'IFN': 2.7750692757047184,
   'GFP': 2.4452932253267043},
  {'IFN': {'IFN': 0.3092617270856581, 'GFP': 0.0},
   'GFP': {'IFN': 0.0, 'GFP': 0.3795830014398247}}),
 '175d6827b300ac2c1ed6ad669cabfc1f': ({'IFN': 2.8281502818385786,
   'GFP': 2.444272632702969},
  {'IFN': {'IFN': 0.32690165198058857, 'GFP': 0.0},
   'GFP': {'IFN': 0.0, 'GFP': 0.37942175406059514}}),
 '10a9c0c205ca6d2dbc16631afaadb620': ({'IFN': 3.409365820211327,
   'GFP': 2.1058467527653146},
  {'IFN': {'IFN': 0.11575594781181384, 'GFP': 0.0},
   'GFP': {'IFN': 0.0, 'GFP': 0.11296469961052302}}),
 'b6783f438d75324c3c9a7728a420fcd4': ({'IFN': 3.2329098347401946,
   'GFP': 2.1016702668109004},
  {'IFN': {'IFN': 0.19526563111384276, 'GFP': 0.0},
   'GFP': {'IFN': 0.0, 'GFP': 0.265969965058789

In [12]:
frontier = compute_posterior_pareto_frontier(
    experiment=ehvi_experiment,
    data=ehvi_experiment.fetch_data(),
    primary_objective=metric_IFN,
    secondary_objective=metric_GFP,
    absolute_metrics=["GFP", "IFN"],
    num_points=5,
)

render(plot_pareto_frontier(frontier, CI_level=0.90)) 

x_IFN= [ 5.8  0.4 13.1 11.4  3.5  8.7  8.5 13.9  8.3  7.3  8.8 10.4]
x_IFN= [10.6  7.8  5.9 11.9 17.7  2.9  9.3  1.  13.6  4.4  3.1 11.9]
x_IFN= [ 0.4  6.2 11.1  5.5  6.6 17.7  1.6 11.8  1.8 17.9  1.4 17.9]
x_IFN= [ 3.2  4.1 18.7 19.7  6.8  0.9  1.3  5.   1.1 17.4  1.  20.7]
x_IFN= [ 9.7  4.7 10.6 14.5  4.9  5.4  5.2  9.1  8.7 14.3 11.2  1.6]
x_IFN= [16.7  4.1  7.6 10.5 12.  13.9  6.9  3.1 12.1  7.1  0.4  5.6]
x_IFN= [ 4.1  2.2  0.1  2.2  4.3 16.1  6.5  8.9 14.2  7.4 16.8 17.2]
x_IFN= [ 4.5 15.3 15.   3.8 11.3  6.   2.3 10.3  2.4 15.1  6.9  7. ]
x_IFN= [ 0.2  7.3  3.3 17.1  5.9 19.7  8.8 17.2  1.9  1.5  4.  13.1]
x_IFN= [17.6 16.1 17.4  3.7  3.6  0.9 15.6  0.1  6.1  1.7 10.3  6.9]
x_IFN= [ 7.5  5.8  3.6 18.8  2.4 14.4 16.8  7.5  3.5  4.   3.  12.7]
x_IFN= [ 9.4  1.1  5.3  5.6 14.8 12.6  8.1  4.9  3.  12.5 14.2  8.5]
x_IFN= [13.3  0.3 15.2 10.6  4.4  3.6  0.7  9.  16.9 13.4 11.4  1.1]
x_IFN= [ 3.  11.7 14.   8.8  0.8  7.5 14.2  5.7 10.6  8.4 11.3  3.9]
x_IFN= [ 4.   1.8 12.3 12.7  0.7 1

[INFO 02-11 12:04:21] ax.modelbridge.transforms.standardize_y: Outcome GFP is constant, within tolerance.
[INFO 02-11 12:04:21] ax.modelbridge.transforms.standardize_y: Outcome IFN is constant, within tolerance.


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [ ]:
for i in range(6,RawData.shape[0]):
    a = [Arm(parameters = {f'x{x+1}': RawData[i,x] for x in range(featureNumber)})]
    gr = GeneratorRun(a)
    trial = ehvi_experiment.new_trial(generator_run=gr)
    trial.run()
    exp_df = exp_to_df(ehvi_experiment)
    outcomes = np.array(exp_df[['IFN', 'GFP']], dtype=np.double)
    try:
        hv = observed_hypervolume(modelbridge=ehvi_model)
    except:
        hv = 0
        print("Failed to compute hv")
    ehvi_hv_list.append(hv)
    print(f"Iteration: {i}, HV: {hv}")

In [ ]:
ehvi_hv_list = []
ehvi_model = None
for i in range(N_BATCH):   
    ehvi_model = get_MOO_EHVI(
        experiment=ehvi_experiment, 
        data=ehvi_data,
    )
    generator_run = ehvi_model.gen(1)
    trial = ehvi_experiment.new_trial(generator_run=generator_run)
    trial.run()
    ehvi_data = Data.from_multiple_data([ehvi_data, trial.fetch_data()])
    exp_df = exp_to_df(ehvi_experiment)
    outcomes = np.array(exp_df[['IFN', 'GFP']], dtype=np.double)
    try:
        hv = observed_hypervolume(modelbridge=ehvi_model)
    except:
        hv = 0
        print("Failed to compute hv")
    ehvi_hv_list.append(hv)
    print(f"Iteration: {i}, HV: {hv}")

ehvi_outcomes = np.array(exp_to_df(ehvi_experiment)[['a', 'b']], dtype=np.double)